In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
import random
from datetime import datetime

import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedKFold
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm
from sklearn.metrics import *
import re
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
! pip install impyute

In [22]:
from impyute.imputation.cs import mice
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [5]:
train_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_err_data.csv')
train_problem = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_problem_data.csv')
test_err = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_err_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/데이콘/data/sample_submission.csv')
train_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/train_quality_data.csv')
test_quality = pd.read_csv('/content/drive/MyDrive/데이콘/data/test_quality_data.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
train_quality_df = train_quality.drop(['time','fwver'],axis=1)
train_quality_df.fillna(0,inplace=True)

def string2num(x):
  x = re.sub(r"[^0-9]+",'',str(x))
  if x == '':
    return 0
  else:
    return int(x)

train_quality_df['quality_5'] = train_quality_df['quality_5'].apply(lambda x : string2num(x))
train_quality_df['quality_7'] = train_quality_df['quality_7'].apply(lambda x : string2num(x))
train_quality_df['quality_8'] = train_quality_df['quality_8'].apply(lambda x : string2num(x))
train_quality_df['quality_9'] = train_quality_df['quality_9'].apply(lambda x : string2num(x))
train_quality_df['quality_10'] = train_quality_df['quality_10'].apply(lambda x : string2num(x))

train_quality_df['quality_5'] = train_quality_df['quality_5'].astype('int')
train_quality_df['quality_7'] = train_quality_df['quality_7'].astype('int')
train_quality_df['quality_8'] = train_quality_df['quality_8'].astype('int')
train_quality_df['quality_9'] = train_quality_df['quality_9'].astype('int')
train_quality_df['quality_10'] = train_quality_df['quality_10'].astype('int')

mean_train_quality = train_quality_df.groupby('user_id').agg('median')
mean_train_quality = mean_train_quality.reset_index()


id_error = train_err[['user_id','errtype']].values
error = np.zeros((15000,42))

for person_idx, err in tqdm(id_error):
  error[person_idx-10000,err-1]+=1

df_error = pd.DataFrame(error)
df_error['user_id'] = train_err['user_id']

merged_train_err_quality = pd.merge(df_error,mean_train_quality,on='user_id',how='left')
del merged_train_err_quality['user_id']

multiple_imputation_df = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(merged_train_err_quality))

problem = np.zeros(15000)
problem[train_problem.user_id.unique()-10000] = 1 

train_x = multiple_imputation_df.values
train_y = problem

smote = SMOTE(random_state=0)
x_train_over, y_train_over = smote.fit_sample(train_x,train_y)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트:',x_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포:\n',pd.Series(y_train_over).value_counts())

In [ ]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

test_quality_df = test_quality.drop(['time','fwver'],axis=1)

test_quality_df['quality_1'] = test_quality_df['quality_1'].apply(lambda x : string2num(x))
test_quality_df['quality_5'] = test_quality_df['quality_5'].apply(lambda x : string2num(x))
test_quality_df['quality_7'] = test_quality_df['quality_7'].apply(lambda x : string2num(x))
test_quality_df['quality_8'] = test_quality_df['quality_8'].apply(lambda x : string2num(x))
test_quality_df['quality_9'] = test_quality_df['quality_9'].apply(lambda x : string2num(x))
test_quality_df['quality_10'] = test_quality_df['quality_10'].apply(lambda x : string2num(x))

test_quality_df['quality_1'] = test_quality_df['quality_1'].astype('int')
test_quality_df['quality_5'] = test_quality_df['quality_5'].astype('int')
test_quality_df['quality_7'] = test_quality_df['quality_7'].astype('int')
test_quality_df['quality_8'] = test_quality_df['quality_8'].astype('int')
test_quality_df['quality_9'] = test_quality_df['quality_9'].astype('int')
test_quality_df['quality_10'] = test_quality_df['quality_10'].astype('int')

test_quality_df.fillna(0,inplace=True)

mean_test_quality = test_quality_df.groupby('user_id').agg('median')
mean_test_quality = mean_test_quality.reset_index()

id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)

df_error_test = pd.DataFrame(test_x)
df_error_test['user_id'] = test_err['user_id']

merged_test_err_quality = pd.merge(df_error_test,mean_test_quality,on='user_id',how='left')
del merged_test_err_quality['user_id']

multiple_imputation_df_test = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(merged_test_err_quality))

In [ ]:
multiple_imputation_df_test

In [32]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

train_y = y_train_over
train_x = x_train_over
test_x = multiple_imputation_df_test.to_numpy()

train_y_t = torch.tensor(train_y, dtype=torch.float32)
train_x_t = torch.tensor(train_x, dtype=torch.float32)
test_x_t = torch.tensor(test_x, dtype=torch.float32)
test_len = len(test_x_t)

N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 48
BATCH_SIZE = 72
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}
prediction = np.zeros((test_len, 1), dtype=np.float32)

for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(train_x, train_y)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(TensorDataset(train_x_t[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(train_x_t[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(test_x_t, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        model = nn.Sequential(
            nn.Dropout(0.05),
            nn.Linear(55, 110, bias=False),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Dropout(0.5),
            nn.Linear(110, 64, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1)
        ).to(DEVICE)
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=7.8e-2)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=N_EPOCH // 6, eta_min=4e-4)
        prediction_t, loss_t = np.zeros((test_len, 1), dtype=np.float32), 1.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad()
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze()
                loss = criterion(pred, yy)
                loss.backward()
                optimizer.step()
                scheduler.step(epoch + idx / len(train_loader))

            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                # print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'
                #       .format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,
                #               running_acc / running_count * 100))

                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        pred = model(xx).detach().to('cpu').numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.4f}'.format(repeat + 1, tot / N_SKFOLD))


01/07: 100%|██████████| 48/48 [01:23<00:00,  1.73s/it]

02/07: 100%|██████████| 48/48 [01:28<00:00,  1.84s/it]

03/07: 100%|██████████| 48/48 [01:26<00:00,  1.81s/it]

04/07: 100%|██████████| 48/48 [01:25<00:00,  1.79s/it]

05/07: 100%|██████████| 48/48 [01:26<00:00,  1.80s/it]

06/07: 100%|██████████| 48/48 [01:27<00:00,  1.82s/it]

07/07: 100%|██████████| 48/48 [01:26<00:00,  1.80s/it]

01/07:   0%|          | 0/48 [00:00<?, ?it/s]

R1 -> 0.5901



01/07: 100%|██████████| 48/48 [01:25<00:00,  1.78s/it]

02/07: 100%|██████████| 48/48 [01:26<00:00,  1.80s/it]

03/07: 100%|██████████| 48/48 [01:25<00:00,  1.78s/it]

04/07: 100%|██████████| 48/48 [01:25<00:00,  1.78s/it]

05/07: 100%|██████████| 48/48 [01:23<00:00,  1.73s/it]

06/07: 100%|██████████| 48/48 [01:26<00:00,  1.79s/it]

07/07: 100%|██████████| 48/48 [01:27<00:00,  1.82s/it]

01/07:   0%|          | 0/48 [00:00<?, ?it/s]

R2 -> 0.5903



01/07: 100%|██████████| 48/48 [01:25<00:00,  1.78s/it]

02/07: 100%|██████████| 48/48 [01:27<00:00,  1.82s/it]

03/07: 100%|██████████| 48/48 [01:25<00:00,  1.79s/it]

04/07: 100%|██████████| 48/48 [01:27<00:00,  1.81s/it]

05/07: 100%|██████████| 48/48 [01:26<00:00,  1.81s/it]

06/07: 100%|██████████| 48/48 [01:27<00:00,  1.82s/it]

07/07: 100%|██████████| 48/48 [01:26<00:00,  1.80s/it]

01/07:   0%|          | 0/48 [00:00<?, ?it/s]

R3 -> 0.5896



01/07: 100%|██████████| 48/48 [01:25<00:00,  1.78s/it]

02/07: 100%|██████████| 48/48 [01:28<00:00,  1.85s/it]

03/07: 100%|██████████| 48/48 [01:27<00:00,  1.81s/it]

04/07: 100%|██████████| 48/48 [01:24<00:00,  1.77s/it]

05/07: 100%|██████████| 48/48 [01:28<00:00,  1.84s/it]

06/07: 100%|██████████| 48/48 [01:24<00:00,  1.76s/it]

07/07: 100%|██████████| 48/48 [01:26<00:00,  1.81s/it]

01/07:   0%|          | 0/48 [00:00<?, ?it/s]

R4 -> 0.5913



01/07: 100%|██████████| 48/48 [01:27<00:00,  1.82s/it]

02/07: 100%|██████████| 48/48 [01:28<00:00,  1.84s/it]

03/07: 100%|██████████| 48/48 [01:27<00:00,  1.81s/it]

04/07: 100%|██████████| 48/48 [01:24<00:00,  1.76s/it]

05/07: 100%|██████████| 48/48 [01:26<00:00,  1.79s/it]

06/07: 100%|██████████| 48/48 [01:26<00:00,  1.80s/it]

07/07: 100%|██████████| 48/48 [01:28<00:00,  1.84s/it]

R5 -> 0.5878


In [33]:
print(max(prediction))
print(min(prediction))

[16.514513]
[-286.0872]


In [34]:
prediction

array([[ 2.0847566 ],
       [-0.12668498],
       [ 0.5132609 ],
       ...,
       [ 1.9840838 ],
       [ 0.61794424],
       [ 0.35201722]], dtype=float32)

In [30]:
sample_submission['problem'] = prediction

In [31]:
sample_submission.to_csv('/content/drive/MyDrive/데이콘/submission/nn_ver1_submission.csv')